In [2]:
import pandas as pd 
import numpy as np 
import requests

### Load Streaming and Library Data 

In [6]:
#Read in our streaming files 
df_stream0 = pd.read_json('Data/StreamingHistory0.json')
df_stream1 = pd.read_json('Data/StreamingHistory1.json')

#merge the streaming dataframes 

df_streams = pd.concat([df_stream0, df_stream1])

df_streams.dtypes

endTime       object
artistName    object
trackName     object
msPlayed       int64
dtype: object

In [13]:
#Create an "UniqueID" by concatenating artistName and trackName

df_streams['UniqueID'] = df_streams['artistName'] + ": " + df_streams['trackName']

df_streams.head()

,endTime,artistName,trackName,msPlayed,UniqueID
0,2020-10-15 00:12,THEY.,What I Know Now (feat. Wiz Khalifa),3715,THEY.: What I Know Now (feat. Wiz Khalifa)
1,2020-10-15 00:12,Sam Smith,My Oasis (feat. Burna Boy),24725,Sam Smith: My Oasis (feat. Burna Boy)
2,2020-10-15 00:12,Sam Smith,My Oasis (feat. Burna Boy),1207,Sam Smith: My Oasis (feat. Burna Boy)
3,2020-10-15 00:15,Cheat Codes,Balenciaga,173706,Cheat Codes: Balenciaga
4,2020-10-15 00:16,ARTY,Rebound - Radio Edit,85740,ARTY: Rebound - Radio Edit


In [16]:
# Read in library data 

df_library = pd.read_json('Data/YourLibrary1.json')

# Created unique column, same as for our stream dataframe

df_library['UniqueID'] = df_library['artist'] + ": " + df_library['track']

split = df_library['uri'].str.split(":", expand=True)

df_library['track_uri'] = split[2]

df_library.head()

,artist,album,track,uri,UniqueID,track_uri
0,Slim,Love's Crazy,So Fly (feat. Yung Joc),spotify:track:2wmBKUx62Px6my6U1mYw18,Slim: So Fly (feat. Yung Joc),2wmBKUx62Px6my6U1mYw18
1,JACKBOYS,JACKBOYS,JACKBOYS,spotify:track:62zKJrpbLxz6InR3tGyr7o,JACKBOYS: JACKBOYS,62zKJrpbLxz6InR3tGyr7o
2,Calvin Harris,18 Months,Thinking About You (feat. Ayah Marar),spotify:track:1KtD0xaLAikgIt5tPbteZQ,Calvin Harris: Thinking About You (feat. Ayah ...,1KtD0xaLAikgIt5tPbteZQ
3,Foo Fighters,Best Of You,Best of You,spotify:track:4nUM7pGcTUK2pY1d2LybrT,Foo Fighters: Best of You,4nUM7pGcTUK2pY1d2LybrT
4,Laidback Luke,Bae (feat. Gina Turner),Bae,spotify:track:4zxhd49IbXASiDuIXmWm5o,Laidback Luke: Bae,4zxhd49IbXASiDuIXmWm5o


In [22]:
#Copy the df_streams to another df - called df_tableau

df_tableau = df_streams.copy()

#Since we created new "UniqueID" for both dataframes, it will be used for our merge

df_tableau =  pd.merge(df_tableau, df_library[['album', 'UniqueID', 'track_uri']], how='left', on=['UniqueID'])

#Drop all songs that arent in our library dataframe 

df_tableau = df_tableau[df_tableau['track_uri'].notna()]
df_tableau.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9239 entries, 0 to 17981
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   endTime     9239 non-null   object
 1   artistName  9239 non-null   object
 2   trackName   9239 non-null   object
 3   msPlayed    9239 non-null   int64 
 4   UniqueID    9239 non-null   object
 5   album       9239 non-null   object
 6   track_uri   9239 non-null   object
dtypes: int64(1), object(6)
memory usage: 577.4+ KB


### Spotify Credentials

In [21]:
# save your IDs from new project in Spotify Developer Dashboard
CLIENT_ID = ''
CLIENT_SECRET = ''

### Generate access token with requests 

In [ ]:
#authentication URL
AUTH_url =- 'https://accounts.spotify.com/api/token'

#POST
auth_response = requests.post(AUTH_url, {
    'grant_type': 'client_credentials',
    'client_id': CLIENT_ID,
    'client_secret':CLIENT_SECRET
})